In [11]:
import numpy as np
import RNTK_NEW
import jax
import symjax
import symjax.tensor as T
import math

In [12]:
L = 2
Ti = 4177
qmatrix = np.zeros([L,Ti],dtype = object)
DATA = T.Placeholder((4177, 8), 'float32')
X = DATA[:,0]
tx = X*X[:, None]
n = tx.shape[0]
su = 1
sb = 1
sh = 1
sq = 1
sw = 1

In [33]:
def normv(x):
    return T.sqrt(T.sum(T.power(x, 2)))
def capQ(lq):
    return T.full((2,2), lq)

In [37]:
for t in range(0, Ti):
    for l in range(0, L):
        print(l, t)
        if ((t == l) & (l == 0)):
            #assuming ||x|| means the norm of that vector
            qmatrix[l, t] = su**2 * normv(X[0])**2 + sb**2 + sh ** 2

        if ((t == 0) & (l > t)):

            S,D = VT(capQ(qmatrix[l-1,t]))
            qmatrix[l, t] = su**2 * S + sb**2

        if ((t > l) & (l == 0)):
            S,D = VT(capQ(qmatrix[l,t-1]))
            qmatrix[l, t] = sw**2 * S + su**2 * normv(X[t])**2 + sb**2

        if ((t > l) & (l > 0)):
            S_left, D_left = VT(capQ(qmatrix[l,t-1]))
            S_right, D_right = VT(capQ(qmatrix[l-1,t]))
            qmatrix[l, t] = sw**2 * S_left + su**2 * D_right + sb**2


0 0
1 0
0 1
1 1
0 2
1 2
0 3
1 3
0 4
1 4
0 5
1 5


KeyboardInterrupt: 

In [15]:
def VT(M):
    A = T.diag(M)  # GP_old is in R^{n*n} having the output gp kernel
    # of all pairs of data in the data set
    B = A * A[:, None]
    C = T.sqrt(B)  # in R^{n*n}
    D = M / C  # this is lamblda in ReLU analyrucal formula
    E = T.clip(D, -1, 1)  # clipping E between -1 and 1 for numerical stability.
    F = (1 / (2 * np.pi)) * (E * (np.pi - T.arccos(E)) + T.sqrt(1 - E ** 2)) * C
    G = (np.pi - T.arccos(E)) / (2 * np.pi)
    return F,G